In [1]:
# download google word2vec pretrained embedding 

!wget -P /content/sample_data -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2020-05-26 12:53:35--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.162.221
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.162.221|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘/content/sample_data/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  16.8MB/s    in 97s     

2020-05-26 12:55:13 (16.2 MB/s) - ‘/content/sample_data/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [14]:
# install NLTK all supporting data and libraries 
!python -m nltk.downloader all

/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipp

In [16]:
!pip install Unidecode

     |████████████████████████████████| 245kB 9.0MB/s 


In [7]:
from gensim.models import KeyedVectors
filename = '/content/sample_data/GoogleNews-vectors-negative300.bin.gz'
model = KeyedVectors.load_word2vec_format(filename, binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [36]:
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from unidecode import unidecode
import string
import numpy as np
from scipy import spatial

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

#remove stop words and punctuations
def pre_process(corpus):
  # convert input corpus to lower case.
  corpus = corpus.lower()
  # collecting a list of stop words from nltk and punctuation form
  # string class and create single array.
  stopset = stopwords.words('english') + list(string.punctuation)
  # remove stop words and punctuations from string.
  # word_tokenize is used to tokenize the input corpus in word tokens.
  corpus = " ".join([i for i in word_tokenize(corpus) if i not in stopset])
  # remove non-ascii characters
  corpus = unidecode(corpus)
  lemmatizer = WordNetLemmatizer()
  words = word_tokenize(corpus)
  processed_words =[]
  for w in words:
    processed_words.append(lemmatizer.lemmatize(w))
  return processed_words
pre_process("Samples of non ASCII: Ceñía. How to remove stopwords and punctuations?")

['sample', 'non', 'ascii', 'cenia', 'remove', 'stopwords', 'punctuation']

In [37]:
# get all words in model
index2word_set = set(model.wv.index2word)

# calculate average vector
def avg_feature_vector(sentence, model, num_features, index2word_set):
    words = pre_process(sentence)
    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in words:
        if word in index2word_set:
            n_words += 1
            feature_vec = np.add(feature_vec, model[word])
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [46]:
s1 = avg_feature_vector("13 oz Ceramic + Stainless Steel Tumbler",model,300,index2word_set)
s2 = avg_feature_vector("20 oz Ceramic-Lined Stainless Steel Tumbler",model,300,index2word_set)
sim = 1 - spatial.distance.cosine(s1, s2)
print(sim)

0.9729264974594116
